In [10]:
from pathlib import Path
from sunpy.net import Fido, attrs as a
from concurrent.futures import ThreadPoolExecutor
import logging

# Set up logging for better debugging and progress tracking
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 1. Create directory to save GOES data
SAVE_DIR = Path("/home/griffingoodwin/downloads/goes_data")
SAVE_DIR.mkdir(exist_ok=True)

# 2. Download GOES data and save to disk
def download_and_save_goes_data(start='2023-07-01', end='2023-08-15', max_workers=4):
    """
    Download GOES X-ray data at 1-minute cadence for a specified time range, including all available satellites.

    Parameters:
    - start (str): Start date for the query (e.g., '2023-07-01')
    - end (str): End date for the query (e.g., '2023-08-15')
    - max_workers (int): Number of parallel download threads
    """
    logging.info(f"Searching GOES X-ray data from {start} to {end} for all satellites at 1-minute cadence...")

    # Query for all GOES satellites with 1-minute averaged XRS data
    goes_query = Fido.search(
        a.Time(start, end),
        a.Instrument('XRS'),
        a.Resolution('avg1m')  # 1-minute averaged data
    )

    logging.info(f"Found {len(goes_query[0])} GOES files.")

    # Skip if no files found
    if len(goes_query[0]) == 0:
        logging.warning("No files found for the specified query.")
        return []

    # Define download function for a single file
    def download_file(file_entry, path_template):
        try:
            result = Fido.fetch(file_entry, path=str(path_template / "{file}"))
            return result
        except Exception as e:
            logging.error(f"Failed to download {file_entry['file']}: {e}")
            return []

    # Use ThreadPoolExecutor for parallel downloads
    logging.info(f"Downloading {len(goes_query[0])} files with {max_workers} workers...")
    downloaded_files = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Create download tasks for each file
        futures = [
            executor.submit(download_file, row, SAVE_DIR)
            for row in goes_query[0]
        ]
        # Collect results
        for future in futures:
            result = future.result()
            downloaded_files.extend(result)

    logging.info(f"Saved {len(downloaded_files)} files to {SAVE_DIR}")
    return downloaded_files



if __name__ == "__main__":
    files = download_and_save_goes_data(
        start='2023-07-01',
        end='2023-08-15',
        max_workers=4
    )

2025-07-08 19:44:50 - root - INFO: Searching GOES X-ray data from 2023-07-01 to 2023-08-15 for all satellites at 1-minute cadence...
2025-07-08 19:44:59 - root - INFO: Found 92 GOES files.
2025-07-08 19:44:59 - root - INFO: Downloading 92 files with 4 workers...
/home/griffingoodwin/.conda/envs/2025-HL-Flaring-MEGS-AI-Updated/lib/python3.10/site-packages/parfive/downloader.py:258: UserWarning: This download has been started in a thread which is not the main thread. You will not be able to interrupt the download.
  warnings.warn(
Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]
sci_xrsf-l2-avg1m_g16_d20230701_v2-2-0.nc:   0%|          | 0.00/3.14M [00:00<?, ?B/s]
sci_xrsf-l2-avg1m_g16_d20230703_v2-2-0.nc:   0%|          | 0.00/3.14M [00:00<?, ?B/s]
sci_xrsf-l2-avg1m_g16_d20230704_v2-2-0.nc:   0%|          | 0.00/3.14M [00:00<?, ?B/s]
sci_xrsf-l2-avg1m_g16_d20230702_v2-2-0.nc:   0%|          | 0.00/3.14M [00:00<?, ?B/s]
sci_xrsf-l2-avg1m_g16_d20230701_v2-2-0.nc:   0%|          |

In [57]:
from pathlib import Path
import logging
import xarray as xr
import pandas as pd

# Setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
SAVE_DIR = Path("/home/griffingoodwin/downloads/goes_data")
OUTPUT_DIR = Path("/home/griffingoodwin/downloads/goes_combined")
OUTPUT_DIR.mkdir(exist_ok=True)

# Global lists to store used files
used_g16_files = []
used_g18_files = []

def combine_goes_data():
    """
    Combine GOES-16 and GOES-18 files and track source files used.
    """
    global used_g16_files, used_g18_files

    g16_files = sorted(SAVE_DIR.glob("*g16*.nc"))
    g18_files = sorted(SAVE_DIR.glob("*g18*.nc"))
    logging.info(f"Found {len(g16_files)} GOES-16 and {len(g18_files)} GOES-18 files")

    def process_files(files, satellite_name, output_file, used_file_list):
        datasets = []
        combined_meta = {}

        for file_path in files:
            try:
                ds = xr.open_dataset(str(file_path))
                datasets.append(ds)
                used_file_list.append(file_path)  # Track file used
                if not combined_meta:
                    combined_meta.update(ds.attrs)
                logging.info(f"Loaded {file_path.name}")
            except Exception as e:
                logging.error(f"Could not load {file_path.name}: {e}")
                continue
            finally:
                if 'ds' in locals():
                    ds.close()

        if not datasets:
            logging.warning(f"No valid datasets for {satellite_name}")
            return

        try:
            combined_ds = xr.concat(datasets, dim='time').sortby('time')
            combined_ds.attrs.update({
                'satellite': satellite_name,
                'instrument': combined_meta.get('instrument', 'XRS'),
                'data_level': combined_meta.get('data_level', 'L2'),
                'resolution': combined_meta.get('resolution', '1-minute'),
                'source': combined_meta.get('source', 'NOAA'),
                'start_time': pd.Timestamp(combined_ds.time.min().values).isoformat(),
                'end_time': pd.Timestamp(combined_ds.time.max().values).isoformat(),
                'source_files': ','.join([f.name for f in used_file_list])
            })

            combined_ds.to_netcdf(str(output_file), engine='netcdf4')
            logging.info(f"Saved combined file: {output_file}")
        except Exception as e:
            logging.error(f"Failed to write {output_file}: {e}")
        finally:
            for ds in datasets:
                ds.close()

    process_files(g16_files, "GOES-16", OUTPUT_DIR / "combined_g16_avg1m.nc", used_g16_files)
    process_files(g18_files, "GOES-18", OUTPUT_DIR / "combined_g18_avg1m.nc", used_g18_files)

def write_used_file_metadata_to_csv():
    """
    Write metadata for only the files used to generate the combined G16 and G18 files.
    """
    OUTPUT_CSV = OUTPUT_DIR / "used_source_file_metadata.csv"
    files = used_g16_files + used_g18_files

    metadata_list = []
    logging.info(f"Extracting metadata from {len(files)} used files...")

    for file_path in files:
        try:
            ds = xr.open_dataset(str(file_path))
            meta = ds.attrs.copy()

            meta['file_name'] = file_path.name
            meta['satellite'] = 'GOES-16' if 'g16' in file_path.name.lower() else 'GOES-18' if 'g18' in file_path.name.lower() else 'Unknown'

            meta.setdefault('instrument', 'XRS')
            meta.setdefault('data_level', 'L2')
            meta.setdefault('resolution', '1-minute')
            meta.setdefault('source', 'NOAA')
            meta.setdefault('start_time', 'Unknown')
            meta.setdefault('end_time', 'Unknown')

            metadata_list.append(meta)
            ds.close()
        except Exception as e:
            logging.error(f"Failed to load metadata from {file_path.name}: {e}")
            continue

    if not metadata_list:
        logging.error("No metadata could be extracted.")
        return

    df = pd.DataFrame(metadata_list)

    # Define standard columns and dynamically add extras
    base_columns = [
        'file_name', 'satellite', 'instrument', 'data_level',
        'resolution', 'source', 'start_time', 'end_time'
    ]
    extra_columns = set().union(*metadata_list) - set(base_columns)
    columns = base_columns + sorted(extra_columns)

    for col in columns:
        if col not in df.columns:
            df[col] = pd.NA

    df = df[columns]

    try:
        df.to_csv(OUTPUT_CSV, index=False)
        logging.info(f"Saved used source file metadata to {OUTPUT_CSV}")
        print("\nUsed source file metadata:")
        print(df.to_string(index=False))
    except Exception as e:
        logging.error(f"Failed to write metadata CSV: {e}")

def save_combined_data_as_csv():
    """
    Save combined GOES-16 and GOES-18 data as CSV files.
    """
    columns_to_interp = ["xrsb_flux", "xrsa_flux"]
    try:
        # Load GOES-16 data and save as CSV
        combined_g16_path = "/home/griffingoodwin/downloads/goes_combined/combined_g16_avg1m.nc"
        combined_g18_path = "/home/griffingoodwin/downloads/goes_combined/combined_g18_avg1m.nc"


        goes16 = xr.open_dataset(combined_g16_path)
        goes16_df = goes16.to_dataframe().reset_index()
        goes16_csv_path = "/home/griffingoodwin/downloads/goes_combined/combined_g16_avg1m.csv"
        goes16_df = goes16_df[goes16_df['quad_diode']==0]

        goes16_df[columns_to_interp] = goes16_df[columns_to_interp].interpolate(method="time", limit_direction="both")
        goes16_df.to_csv(goes16_csv_path, index=False)

        # Load GOES-18 data and save as CSV
        goes18 = xr.open_dataset(combined_g18_path)
        goes18_df = goes18.to_dataframe().reset_index()
        goes18_csv_path = "/home/griffingoodwin/downloads/goes_combined/combined_g18_avg1m.csv"
        goes18_df = goes18_df[goes18_df['quad_diode']==0]

        goes18_df[columns_to_interp] = goes18_df[columns_to_interp].interpolate(method="time", limit_direction="both")
        goes18_df.to_csv(goes18_csv_path, index=False)
    except Exception as e:
        print(f"Failed to save combined data as CSV: {e}")

# Run everything
if __name__ == "__main__":
    combine_goes_data()
    write_used_file_metadata_to_csv()
    save_combined_data_as_csv()

2025-07-08 21:54:04,363 - INFO - Found 46 GOES-16 and 46 GOES-18 files
2025-07-08 21:54:04,389 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230701_v2-2-0.nc
2025-07-08 21:54:04,415 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230702_v2-2-0.nc
2025-07-08 21:54:04,440 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230703_v2-2-0.nc
2025-07-08 21:54:04,468 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230704_v2-2-0.nc
2025-07-08 21:54:04,661 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230705_v2-2-0.nc
2025-07-08 21:54:04,686 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230706_v2-2-0.nc
2025-07-08 21:54:04,712 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230707_v2-2-0.nc
2025-07-08 21:54:04,737 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230708_v2-2-0.nc
2025-07-08 21:54:04,763 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230709_v2-2-0.nc
2025-07-08 21:54:04,789 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230710_v2-2-0.nc
2025-07-08 21:54:04,814 - INFO - Loaded sci_xrsf-l2-avg1m_g16_d20230711_v2-2-0.nc
2025-07-08 21:54:04,839 - I


Used source file metadata:
                                file_name satellite                                                                   instrument data_level resolution                           source start_time end_time            Conventions L1b_production_site L1b_system_environment     algorithm algorithm_date algorithm_parameters algorithm_version       creator_email            creator_institution creator_name creator_type                creator_url             date_created                          git_commit_hash                    history                                        id                                                                    input_files_first                                                                     input_files_last  input_files_total input_system_environments     institution instrument_id                                 keywords                                                                                                              

In [1]:
from pathlib import Path

import numpy as np
from sunpy.net import Fido, attrs as a
from concurrent.futures import ThreadPoolExecutor
import logging
import xarray as xr
import pandas as pd

class SXRDownloader:
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    def __init__(self, save_dir: str = '/downloads/goes_data', concat_dir: str = '/downloads/goes_combined'):
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(exist_ok=True)
        self.concat_dir = Path(concat_dir)
        self.concat_dir.mkdir(exist_ok=True)
        self.used_g13_files = []
        self.used_g14_files = []
        self.used_g15_files = []
        self.used_g16_files = []
        self.used_g17_files = []
        self.used_g18_files = []


    def download_and_save_goes_data(self, start='2023-07-01', end='2023-08-15', max_workers=4):
        """
        Download GOES X-ray data at 1-minute cadence for a specified time range, including all available satellites.

        Parameters:
        - start (str): Start date for the query (e.g., '2023-07-01')
        - end (str): End date for the query (e.g., '2023-08-15')
        - max_workers (int): Number of parallel download threads
        """
        logging.info(f"Searching GOES X-ray data from {start} to {end} for all satellites at 1-minute cadence...")

        # Query for all GOES satellites with 1-minute averaged XRS data
        goes_query = Fido.search(
            a.Time(start, end),
            a.Instrument('XRS'),
            a.Resolution('avg1m')  # 1-minute averaged data
        )

        logging.info(f"Found {len(goes_query[0])} GOES files.")

        # Skip if no files found
        if len(goes_query[0]) == 0:
            logging.warning("No files found for the specified query.")
            return []

        # Define download function for a single file
        def download_file(file_entry, path_template):
            try:
                fido_result = Fido.fetch(file_entry, path=str(path_template / "{file}"))
                return fido_result
            except Exception as e:
                logging.error(f"Failed to download {file_entry['file']}: {e}")
                return []

        # Use ThreadPoolExecutor for parallel downloads
        logging.info(f"Downloading {len(goes_query[0])} files with {max_workers} workers...")
        downloaded_files = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Create download tasks for each file
            futures = [
                executor.submit(download_file, row, self.save_dir)
                for row in goes_query[0]
            ]
            # Collect results
            for future in futures:
                result = future.result()
                downloaded_files.extend(result)

        logging.info(f"Saved {len(downloaded_files)} files to {self.save_dir}")
        return downloaded_files


    def combine_goes_data(self, columns_to_interp=["xrsb_flux", "xrsa_flux"]):
        """
        Combine GOES-16 and GOES-18 files and track source files used.
        Parameters
        """

        g13_files = sorted(self.save_dir.glob("*g13*.nc"))
        g14_files = sorted(self.save_dir.glob("*g14*.nc"))
        g15_files = sorted(self.save_dir.glob("*g15*.nc"))
        g16_files = sorted(self.save_dir.glob("*g16*.nc"))
        g17_files = sorted(self.save_dir.glob("*g17*.nc"))
        g18_files = sorted(self.save_dir.glob("*g18*.nc"))
        logging.info(f"Found {len(g13_files)} GOES-13 files, {len(g14_files)} GOES-14 files, {len(g15_files)} GOES-15 files, {len(g16_files)} GOES-16 files, {len(g17_files)} GOES-17 files, and {len(g18_files)} GOES-18 files.")

        def process_files(files, satellite_name, output_file, used_file_list):
            datasets = []
            combined_meta = {}

            for file_path in files:
                try:
                    ds = xr.open_dataset(str(file_path))
                    datasets.append(ds)
                    used_file_list.append(file_path)  # Track file used
                    logging.info(f"Loaded {file_path.name}")
                except Exception as e:
                    logging.error(f"Could not load {file_path.name}: {e}")
                    continue
                finally:
                    if 'ds' in locals():
                        ds.close()

            if not datasets:
                logging.warning(f"No valid datasets for {satellite_name}")
                return

            try:
                combined_ds = xr.concat(datasets, dim='time').sortby('time')
                if satellite_name in ['GOES-13', 'GOES-14', 'GOES-15']:
                    combined_ds['xrsa_flux'] = combined_ds['xrsa_flux'] / .85
                    combined_ds['xrsb_flux'] = combined_ds['xrsb_flux'] / .7
                df = combined_ds.to_dataframe().reset_index()
                if 'quad_diode' in df.columns:
                    df = df[df['quad_diode'] == 0]  # Filter out quad diode data
                df['time'] = pd.to_datetime(df['time'])
                df.set_index('time', inplace=True)
                df_log = np.log10(df[columns_to_interp].replace(0, np.nan))

                # Step 3: Interpolate in log space
                df_log_interp = df_log.interpolate(method="time", limit_direction="both")

                # Step 4: Back-transform to linear space
                df[columns_to_interp] = 10 ** df_log_interp

                # Add min and max dates to filename
                min_date = df.index.min().strftime('%Y%m%d')
                max_date = df.index.max().strftime('%Y%m%d')
                filename = f"{str(output_file)}_{min_date}_{max_date}.csv"
                df.to_csv(filename, index=True)




                logging.info(f"Saved combined file: {output_file}")
            except Exception as e:
                logging.error(f"Failed to write {output_file}: {e}")
            finally:
                for ds in datasets:
                    ds.close()
        if len(g13_files) != 0:

            process_files(g13_files, "GOES-13", self.concat_dir / "combined_g13_avg1m", self.used_g13_files)
        if len(g14_files) != 0:
            process_files(g14_files, "GOES-14", self.concat_dir / "combined_g14_avg1m", self.used_g14_files)
        if len(g15_files) != 0:
            process_files(g15_files, "GOES-15", self.concat_dir / "combined_g15_avg1m", self.used_g15_files)
        if len(g16_files) != 0:
            process_files(g16_files, "GOES-16", self.concat_dir / "combined_g16_avg1m", self.used_g16_files)
        if len(g17_files) != 0:
            process_files(g17_files, "GOES-17", self.concat_dir / "combined_g17_avg1m", self.used_g17_files)
        if len(g18_files) != 0:
            process_files(g18_files, "GOES-18", self.concat_dir / "combined_g18_avg1m", self.used_g18_files)

/home/griffingoodwin/.conda/envs/2025-HL-Flaring-MEGS-AI-Updated/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
sx = SXRDownloader('/home/griffingoodwin/downloads/goes_data', '/home/griffingoodwin/downloads/goes_combined')

In [169]:
sx.download_and_save_goes_data('2023-07-01', '2023-08-15', max_workers=4)

2025-07-08 23:24:16,148 - INFO - Searching GOES X-ray data from 2023-07-01 to 2023-08-15 for all satellites at 1-minute cadence...
2025-07-08 23:24:19,222 - INFO - Found 92 GOES files.
2025-07-08 23:24:19,223 - INFO - Downloading 92 files with 4 workers...
/home/griffingoodwin/.conda/envs/2025-HL-Flaring-MEGS-AI-Updated/lib/python3.10/site-packages/parfive/downloader.py:258: UserWarning: This download has been started in a thread which is not the main thread. You will not be able to interrupt the download.
  warnings.warn(


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

...

sci_xrsf-l2-avg1m_g18_d20230813_v2-2-0.nc:   0%|          | 0.00/3.14M [00:00<?, ?B/s]

sci_xrsf-l2-avg1m_g18_d20230814_v2-2-0.nc:   0%|          | 0.00/3.14M [00:00<?, ?B/s]

sci_xrsf-l2-avg1m_g18_d20230815_v2-2-0.nc:   0%|          | 0.00/3.14M [00:00<?, ?B/s]

2025-07-08 23:24:22,910 - INFO - Saved 92 files to /home/griffingoodwin/downloads/goes_data


['/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230701_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230702_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230703_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230704_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230705_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230706_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230707_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230708_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230709_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230710_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/sci_xrsf-l2-avg1m_g16_d20230711_v2-2-0.nc',
 '/home/griffingoodwin/downloads/goes_data/

In [3]:
sx.combine_goes_data()

2025-07-09 17:12:13 - root - INFO: Found 0 GOES-13 files, 0 GOES-14 files, 0 GOES-15 files, 46 GOES-16 files, 0 GOES-17 files, and 46 GOES-18 files.
2025-07-09 17:12:15 - root - INFO: Loaded sci_xrsf-l2-avg1m_g16_d20230701_v2-2-0.nc
2025-07-09 17:12:15 - root - INFO: Loaded sci_xrsf-l2-avg1m_g16_d20230702_v2-2-0.nc
2025-07-09 17:12:15 - root - INFO: Loaded sci_xrsf-l2-avg1m_g16_d20230703_v2-2-0.nc
2025-07-09 17:12:15 - root - INFO: Loaded sci_xrsf-l2-avg1m_g16_d20230704_v2-2-0.nc
2025-07-09 17:12:15 - root - INFO: Loaded sci_xrsf-l2-avg1m_g16_d20230705_v2-2-0.nc
2025-07-09 17:12:15 - root - INFO: Loaded sci_xrsf-l2-avg1m_g16_d20230706_v2-2-0.nc
2025-07-09 17:12:15 - root - INFO: Loaded sci_xrsf-l2-avg1m_g16_d20230707_v2-2-0.nc
2025-07-09 17:12:15 - root - INFO: Loaded sci_xrsf-l2-avg1m_g16_d20230708_v2-2-0.nc
2025-07-09 17:12:15 - root - INFO: Loaded sci_xrsf-l2-avg1m_g16_d20230709_v2-2-0.nc
2025-07-09 17:12:15 - root - INFO: Loaded sci_xrsf-l2-avg1m_g16_d20230710_v2-2-0.nc
2025-07-09 

In [160]:
c = pd.read_csv('/home/griffingoodwin/downloads/goes_combined/combined_g14_avg1m.csv')

In [161]:
max(c['xrsa_flux'])

1.1648506e-08

In [162]:
min(c['xrsa_flux'])

1e-12

In [124]:
 c['xrsa_flux']= c['xrsa_flux']/1000

In [132]:
c['xrsb_flux']

<xarray.DataArray 'xrsb_flux' (time: 59040)> Size: 236kB
[59040 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 472kB 2012-01-01 ... 2015-01-05T23:59:00
Attributes:
    long_name:            XRS-B primary average flux.
    comments:             Electron contamination has been removed. xrsb_flux ...
    units:                W/m2
    valid_min:            1e-09
    valid_max:            0.2
    ancillary_variables:  xrsb_flag xrsb_num xrsb_flag_excluded

In [133]:
c['xrsa_flux']

<xarray.DataArray 'xrsa_flux' (time: 59040)> Size: 236kB
array([1.0000000e-12, 1.4172792e-12, 1.5865039e-12, ..., 2.7605548e-11,
       2.5108653e-11, 2.6723311e-11], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 472kB 2012-01-01 ... 2015-01-05T23:59:00

In [4]:
from sunpy.net import Fido, attrs as a

goes_query = Fido.search(
    a.Time("2017-09-11 05:15:00", "2017-09-11 05:45:00"),
    a.Instrument('XRS'),
    a.Resolution('avg1m'))

In [5]:
print(goes_query)

Results from 1 Provider:

3 Results from the XRSClient:
Source: <8: https://umbra.nascom.nasa.gov/goes/fits 
8-15: https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/science/ 
16-17: https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/

       Start Time               End Time        Instrument  Physobs   Source Provider Resolution SatelliteNumber filename_res
----------------------- ----------------------- ---------- ---------- ------ -------- ---------- --------------- ------------
2017-09-11 00:00:00.000 2017-09-11 23:59:59.999        XRS irradiance   GOES     NOAA      avg1m              16           --
2017-09-11 00:00:00.000 2017-09-11 23:59:59.999        XRS irradiance   GOES     NOAA      avg1m              13         xrsf
2017-09-11 00:00:00.000 2017-09-11 23:59:59.999        XRS irradiance   GOES     NOAA      avg1m              15         xrsf




In [15]:
result = Fido.search(a.Time('2012/3/4', '2012/3/6'), a.Instrument.lyra, a.Level.two)


In [16]:
print(result)

Results from 1 Provider:

3 Results from the LYRAClient:
Source: http://proba2.oma.be/lyra/data/bsd

       Start Time               End Time        Instrument ... Provider Level
----------------------- ----------------------- ---------- ... -------- -----
2012-03-04 00:00:00.000 2012-03-04 23:59:59.999       LYRA ...      ESA     2
2012-03-05 00:00:00.000 2012-03-05 23:59:59.999       LYRA ...      ESA     2
2012-03-06 00:00:00.000 2012-03-06 23:59:59.999       LYRA ...      ESA     2


